In [1]:
# import
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import inflection
import boto3
import unittest
import os

# config
data_path = '/home/chc/data'

# Question 3

In [2]:
# Download data
question3_data_path = data_path + '/jobposts'
question3_data_file = question3_data_path +'/data job posts.csv'

if not os.path.exists(question3_data_path):
    os.mkdir(question3_data_path)

if not os.path.exists(question3_data_file):
    print('Downloading data as '+question3_data_file)
    os.system('cd '+question3_data_path+';kg dataset -o madhab -d jobposts')
else:
    print('Data file exist '+question3_data_file)
    
# Some issue with the csv file as is in zip format but the extension was wrong
# Have to manually change it as zip and run unzip command
    
# Load data up
question3_data = pd.read_csv(question3_data_file)
question3_data.head()    

Data file exist /home/chc/data/jobposts/data job posts.csv


,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


b) Extract the following fields from the jobpost column:

In [3]:
section_b = pd.DataFrame()
section_b['Company'] = question3_data['Company']
section_b['Job Title'] = question3_data['jobpost'].str.extract('(TITLE:\s{1,}.*\w)', expand=False).str.replace('(TITLE:\s{1,})','')
section_b['Position Duration'] = question3_data['jobpost'].str.extract('(DURATION:\s{1,}.*\w)', expand=False).str.replace('(DURATION:\s{1,})','')
section_b['Position Location'] = question3_data['jobpost'].str.extract('(LOCATION:\s{1,}.*\w)', expand=False).str.replace('(LOCATION:\s{1,})','')
section_b['Job Description'] = question3_data['jobpost'].str.extract('(DESCRIPTION:\s{1,}.*\w)', expand=False).str.replace('(DESCRIPTION:\s{1,})','')
section_b['Job Responsibilities'] = question3_data['jobpost'].str.extract('(RESPONSIBILITIES:\s{1,}.*\w)', expand=False).str.replace('(RESPONSIBILITIES:\s{1,})','')
section_b['Required Qualifications'] = question3_data['jobpost'].str.extract('(QUALIFICATIONS:\s{1,}.*\w)', expand=False).str.replace('(QUALIFICATIONS:\s{1,})','')
section_b['Remuneration'] = question3_data['jobpost'].str.extract('(REMUNERATION:\s{1,}.*\w)', expand=False).str.replace('(REMUNERATION:\s{1,})','')
section_b['Application Deadline'] = question3_data['jobpost'].str.extract('(DEADLINE:\s{1,}.*\w)', expand=False).str.replace('(DEADLINE:\s{1,})','')
section_b['About Company'] = question3_data['jobpost'].str.extract('(COMPANY:\s{1,}.*\w)', expand=False).str.replace('(COMPANY:\s{1,})','')
section_b.head()

,Company,Job Title,Position Duration,Position Location,Job Description,Job Responsibilities,Required Qualifications,Remuneration,Application Deadline,About Company
0,AMERIA Investment Consulting Company,Chief Financial Officer,NaN,"Yerevan, Armenia",AMERIA Investment Consulting Company is seeking a,- Supervises financial management and administ...,"To perform this job successfully, an",NaN,26 January 2004,NaN
1,International Research & Exchanges Board (IREX),Full-time Community Connections Intern (paid i...,3 months,"IREX Armenia Main Office; Yerevan, Armenia",IREX currently seeks to fill the position of a...,- Presenting the CC program to interested parties,- Bachelor's Degree; Master's is preferred,Commensurate with experience,12 January 2004,The International Research & Exchanges Board (...
2,Caucasus Environmental NGO Network (CENN),Country Coordinator,Renewable annual contract,"Yerevan, Armenia",Public outreach and strengthening of a growing,- Working with the Country Director to provide...,"- Degree in environmentally related field, or ...",Salary commensurate with experience,20 January 2004,The Caucasus Environmental NGO Network is a
3,Manoff Group,BCC Specialist,NaN,"Manila, Philippines",The LEAD (Local Enhancement and Development for,- Identify gaps in knowledge and overseeing in...,"- Advanced degree in public health, social sci...",NaN,23 January 2004,NaN
4,Yerevan Brandy Company,Software Developer,NaN,"Yerevan, Armenia",NaN,- Rendering technical assistance to Database M...,- University degree; economical background is ...,Will be commensurate with the norms accepted i...,"20 January 2004, 18:00",NaN


c) Identify the company with the most number of job ads in the past 2 years

In [4]:
question3_data.groupby('Company')['jobpost'].agg('count').idxmax()

'ArmenTel CJSC'

d) Identify the month with the largest number of job ads over the years

In [5]:
def convertToTime(x):
    if type(x) == pd.Timestamp:
        return x.strftime('%Y%m')
    else:
        return '-'

section_d = pd.DataFrame()
section_d['Company'] = question3_data['Company']
# Format is as YearMonth as we will consider year as well besides month
section_d['Month Year'] = pd.to_datetime(question3_data['date'],format='%b %d, %Y',errors='coerce').map \
                            (lambda x: convertToTime(x))
# Remove unknown date format that doesn't have year - eg Jun 1 10:13 PM
section_d = section_d[section_d['Month Year'] != '-']
    
section_d.groupby(['Company','Month Year'])['Month Year'].agg('count').idxmax()

('Orange Armenia', '200907')

e) Clean text and generate new text from Job Responsibilities column: The new text
shall not contain any stop words, and the plural words shall be converted into
singular words.

In [6]:
nltk.download('stopwords')

stop = stopwords.words('english')

def custom_stop(text):    
    if pd.isnull(text):
        return ''
    else:
        return ' '.join([word for word in text.split() if word not in (stop)])

def custom_singularize(text):
    if pd.isnull(text):
        return ''
    else:
        return inflection.singularize(text)
    

remove_stop_words = lambda r:[[word for word in word_tokenize(sente) if word not in stop_words] for sente in sent_tokenize(r)]

section_b['Job Responsibilities'] = section_b['Job Responsibilities'].apply(custom_stop)
section_b['Job Responsibilities'] = section_b['Job Responsibilities'].apply(custom_singularize)

section_b.head()

[nltk_data] Downloading package stopwords to /home/chc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Company,Job Title,Position Duration,Position Location,Job Description,Job Responsibilities,Required Qualifications,Remuneration,Application Deadline,About Company
0,AMERIA Investment Consulting Company,Chief Financial Officer,NaN,"Yerevan, Armenia",AMERIA Investment Consulting Company is seeking a,- Supervises financial management administrati...,"To perform this job successfully, an",NaN,26 January 2004,NaN
1,International Research & Exchanges Board (IREX),Full-time Community Connections Intern (paid i...,3 months,"IREX Armenia Main Office; Yerevan, Armenia",IREX currently seeks to fill the position of a...,- Presenting CC program interested party,- Bachelor's Degree; Master's is preferred,Commensurate with experience,12 January 2004,The International Research & Exchanges Board (...
2,Caucasus Environmental NGO Network (CENN),Country Coordinator,Renewable annual contract,"Yerevan, Armenia",Public outreach and strengthening of a growing,- Working Country Director provide environment...,"- Degree in environmentally related field, or ...",Salary commensurate with experience,20 January 2004,The Caucasus Environmental NGO Network is a
3,Manoff Group,BCC Specialist,NaN,"Manila, Philippines",The LEAD (Local Enhancement and Development for,- Identify gaps knowledge overseeing informati...,"- Advanced degree in public health, social sci...",NaN,23 January 2004,NaN
4,Yerevan Brandy Company,Software Developer,NaN,"Yerevan, Armenia",NaN,- Rendering technical assistance Database Mana...,- University degree; economical background is ...,Will be commensurate with the norms accepted i...,"20 January 2004, 18:00",NaN


f) Write functions to identify null/NA values and to replace null/NA values with a
custom message in “Duration” column

In [7]:
section_b['Duration'] = section_b['Position Duration']
section_b['Duration'].fillna('-', inplace=True)
section_b.isnull().sum()        

Company                        7
Job Title                     23
Position Duration           8210
Position Location             47
Job Description               60
Job Responsibilities           0
Required Qualifications      476
Remuneration               18952
Application Deadline          57
About Company               6531
Duration                       0
dtype: int64

g) Store the results in a new Dataframe/SQL table(s)

In [8]:
section_g = section_b

h) Write the results to an S3 bucket

In [9]:
# Create an S3 client
s3 = boto3.client('s3')

filename = 'q3_upload.csv'
bucket_name = 'seekchc'

# Write to file
section_g.to_csv(data_path+'/'+filename, index=False)

# S3 upload
s3.upload_file(data_path+'/'+filename, bucket_name, filename)